```{contents}
```
## **Agent Role in LangGraph**

In LangGraph, an **agent role** defines the **functional responsibility, authority, and behavioral policy** of an autonomous component inside a multi-agent graph.
Roles are not cosmetic labels — they **shape control flow, memory access, decision rights, and execution scope** of each agent.

---

### **1. Why Agent Roles Are Necessary**

Large LLM systems become unstable without explicit specialization.
LangGraph enforces **structured cooperation** through roles.

| Problem Without Roles  | With Roles                  |
| ---------------------- | --------------------------- |
| Unclear responsibility | Clear functional boundaries |
| Unstable decisions     | Predictable control         |
| Poor scalability       | Modular expansion           |
| Hard to debug          | Isolated agent traces       |

---

### **2. Core Agent Role Categories**

| Role                  | Responsibility                |
| --------------------- | ----------------------------- |
| **Supervisor**        | Orchestrates overall workflow |
| **Planner**           | Decomposes goals into steps   |
| **Executor**          | Performs concrete actions     |
| **Researcher**        | Gathers external knowledge    |
| **Critic / Reviewer** | Evaluates correctness         |
| **Verifier**          | Validates against constraints |
| **Tool Agent**        | Interfaces with tools         |
| **Memory Agent**      | Manages knowledge storage     |
| **Human Proxy**       | Represents human decisions    |

Each role maps to **dedicated nodes or subgraphs**.

---

### **3. Role as Behavioral Contract**

An agent role specifies:

| Dimension     | Meaning                       |
| ------------- | ----------------------------- |
| Scope         | What it is allowed to do      |
| Authority     | What decisions it can make    |
| Memory Access | Which state it can read/write |
| Tools         | Which tools it can call       |
| Policies      | Safety and quality rules      |

This is enforced by **graph topology + node constraints**.

---

### **4. Typical Multi-Agent Topology**

```
Goal
  ↓
Planner Agent
  ↓
Supervisor Agent
  ↓
 ┌──────────┬──────────┬──────────┐
Researcher  Executor   Critic
   ↓           ↓         ↓
            Verifier
                ↓
             Final Answer
```

---

### **5. Implementation Example**

```python
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    goal: str
    plan: str
    result: str
    critique: str

def planner(state):
    return {"plan": f"Steps to achieve: {state['goal']}"}

def executor(state):
    return {"result": f"Executed: {state['plan']}"}

def critic(state):
    return {"critique": "Result looks correct."}

builder = StateGraph(State)

builder.add_node("Planner", planner)
builder.add_node("Executor", executor)
builder.add_node("Critic", critic)

builder.set_entry_point("Planner")
builder.add_edge("Planner", "Executor")
builder.add_edge("Executor", "Critic")
builder.add_edge("Critic", END)

graph = builder.compile()
graph.invoke({"goal": "Write report"})
```

Each node corresponds to a **distinct agent role**.

---

### **6. Role-Based Control Flow**

LangGraph enforces role sequencing through edges:

```python
builder.add_edge("Supervisor", "Researcher")
builder.add_edge("Researcher", "Executor")
builder.add_edge("Executor", "Critic")
```

Conditional routing allows **dynamic delegation**:

```python
def route(state):
    if "error" in state["result"]:
        return "Researcher"
    return "Critic"
```

---

### **7. Role Isolation & Safety**

| Feature                 | Purpose                     |
| ----------------------- | --------------------------- |
| Separate memory scopes  | Prevent interference        |
| Restricted tool access  | Enforce policies            |
| Independent checkpoints | Debug per role              |
| Auditable traces        | Compliance & explainability |

---

### **8. Advanced Role Patterns**

| Pattern               | Description                     |
| --------------------- | ------------------------------- |
| Hierarchical Roles    | Supervisor → Sub-supervisors    |
| Swarm Roles           | Many workers, no central leader |
| Dynamic Roles         | Role assigned at runtime        |
| Debate Roles          | Proponent vs Opponent           |
| Self-Reflective Roles | Executor + Critic loop          |

---

### **9. Why LangGraph Roles Matter**

Agent roles turn LLMs from **chatbots** into **governed distributed systems**:

> **Role = contract + control + accountability**

This enables enterprise-grade multi-agent architectures.


### Demonstration

In [5]:
from typing import TypedDict

class State(TypedDict):
    goal: str
    plan: str
    result: str
    critique: str
    approved: bool

def planner_agent(state: State):
    return {
        "plan": f"1. Identify product value. 2. Write concise summary for: {state['goal']}"
    }

def executor_agent(state: State):
    return {
        "result": f"Launch Summary: Introducing our new AI product that boosts productivity."
    }

def critic_agent(state: State):
    critique = "Looks good." if "Introducing" in state["result"] else "Needs improvement."
    return {
        "critique": critique,
        "approved": critique == "Looks good."
    }

def supervisor_router(state: State):
    if not state.get("plan"):
        return "Planner"
    if not state.get("result"):
        return "Executor"
    if not state.get("approved"):
        return "Critic"
    return "end"



from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("Planner", planner_agent)
builder.add_node("Executor", executor_agent)
builder.add_node("Critic", critic_agent)

builder.set_entry_point("Planner")

builder.add_edge("Planner", "Executor")
builder.add_edge("Executor", "Critic")

builder.add_conditional_edges(
    "Critic",
    supervisor_router,
    {
        "Planner": "Planner",
        "Executor": "Executor",
        "Critic": "Critic",
        "end": END,
    }
)

graph = builder.compile()


In [6]:
final_state = graph.invoke({"goal": "Launch our AI assistant"})
print(final_state)


{'goal': 'Launch our AI assistant', 'plan': '1. Identify product value. 2. Write concise summary for: Launch our AI assistant', 'result': 'Launch Summary: Introducing our new AI product that boosts productivity.', 'critique': 'Looks good.', 'approved': True}
